In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from glob import glob
import pydicom
import torch 
from torch.nn import functional as F
import os
from albumentations.pytorch import ToTensorV2
import albumentations as A
from tqdm.auto import tqdm

import matplotlib.pyplot as plt

from evaluation import get_models, get_volume, preprocess_stage1_sagittal, prediction_stage1_sagittal


/home/jupyter/projects/Kosei-Line/rsna-2024-lumbar-spine-degenerative-classification/src/stage2/exp076/evaluation.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.57617188 0.58203125 0.57617188 ... 0.56835938 0.5625     0.53710938]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  axial_keypoints_df.loc[:, ["x_l", "x_r", "y_l", "y_r"]] = (
/home/jupyter/projects/Kosei-Line/rsna-2024-lumbar-spine-degenerative-classification/src/stage2/exp076/evaluation.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.45898438 0.47070312 0.47265625 ... 0.45507812 0.44726562 0.4296875 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  axial_keypoints_df.loc[:, ["x_l", "x_r", "y_l", "y_r"]] = (
/home/jupyter/projects/Kosei-Line/rsna-2024-lumbar-spine-degenerative-classifi

In [2]:
series_description_df = pd.read_csv(
    "../../../input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv"
)
train_df = pd.read_csv(
    "../../../input/rsna-2024-lumbar-spine-degenerative-classification/train.csv"
)
train_df["fold_id"] = -1
for i, (train_index, valid_index) in enumerate(
    GroupKFold(n_splits=5).split(
        train_df, np.arange(len(train_df)), train_df.study_id
    )
):
    train_df.loc[valid_index, "fold_id"] = i
series_description_df = series_description_df.merge(
    train_df.loc[:, ["study_id", "fold_id"]], on=["study_id"], how="left"
)

In [3]:
series_description_df

,study_id,series_id,series_description,fold_id
0,4003253,702807833,Sagittal T2/STIR,4
1,4003253,1054713880,Sagittal T1,4
2,4003253,2448190387,Axial T2,4
3,4646740,3201256954,Axial T2,1
4,4646740,3486248476,Sagittal T1,1
...,...,...,...,...
6289,4287160193,1507070277,Sagittal T2/STIR,3
6290,4287160193,1820446240,Axial T2,3
6291,4290709089,3274612423,Sagittal T2/STIR,0
6292,4290709089,3390218084,Axial T2,0


In [4]:
# device = "cpu"
device = "cuda:0"
sagittal_t1_result = []
sagittal_t2_result = []
for fold in range(5):
    valid_df = series_description_df[series_description_df.fold_id == fold].reset_index()
    model_stage1_sagittal_t1 = get_models(stage="1", exp_id="017", series_description="Sagittal_T1", fold=fold, logdirs=None, device=device)
    model_stage1_sagittal_t2 = get_models(stage="1", exp_id="017", series_description="Sagittal_T2-STIR", fold=fold, logdirs=None, device=device)
    for series_id in tqdm(valid_df[valid_df.series_description == "Sagittal T1"].series_id.unique()):
        volume = get_volume(series_id=series_id, description="Sagittal T1")
        stage1_volume = preprocess_stage1_sagittal(volume).to(device)
        keypoints = prediction_stage1_sagittal(model_stage1_sagittal_t1, stage1_volume)
        for i, level in enumerate((["L1/L2", "L2/L3", "L3/L4", "L4/L5", "L5/S1"])):
            sagittal_t1_result.append([series_id, level, keypoints[i][0], keypoints[i][1]])
        # break
    for series_id in tqdm(valid_df[valid_df.series_description == "Sagittal T2/STIR"].series_id.unique()):
        volume = get_volume(series_id=series_id, description="Sagittal T2/STIR")
        stage1_volume = preprocess_stage1_sagittal(volume).to(device)
        keypoints = prediction_stage1_sagittal(model_stage1_sagittal_t2, stage1_volume)
        for i, level in enumerate((["L1/L2", "L2/L3", "L3/L4", "L4/L5", "L5/S1"])):
            sagittal_t2_result.append([series_id, level, keypoints[i][0], keypoints[i][1]])
        # break

loading ckpt:  ../../../logs/stage1/exp017/swinv2_tiny_unet_scse_20x256x256_mixup/Sagittal_T1/fold0/rsna-2024-lumbar-spine-degenerative-classification/qbpycslj/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/convnext_tiny_unet_scse_20x256x256_mixup/Sagittal_T1/fold0/rsna-2024-lumbar-spine-degenerative-classification/4zr73olk/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/caformer_s18_unet_scse_20x256x256_mixup/Sagittal_T1/fold0/rsna-2024-lumbar-spine-degenerative-classification/9n64gq36/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/resnetrs50_unet_scse_20x256x256_mixup/Sagittal_T1/fold0/rsna-2024-lumbar-spine-degenerative-classification/exhkxc9i/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/swinv2_tiny_unet_scse_20x256x256_mixup/Sagittal_T2-STIR/fold0/rsna-2024-lumbar-spine-degenerative-classification/wb4vx4bd/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/convnext_tiny_u

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/395 [00:00<?, ?it/s]

loading ckpt:  ../../../logs/stage1/exp017/swinv2_tiny_unet_scse_20x256x256_mixup/Sagittal_T1/fold1/rsna-2024-lumbar-spine-degenerative-classification/4mr2cq2i/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/convnext_tiny_unet_scse_20x256x256_mixup/Sagittal_T1/fold1/rsna-2024-lumbar-spine-degenerative-classification/g746po7r/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/caformer_s18_unet_scse_20x256x256_mixup/Sagittal_T1/fold1/rsna-2024-lumbar-spine-degenerative-classification/ks7555bg/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/resnetrs50_unet_scse_20x256x256_mixup/Sagittal_T1/fold1/rsna-2024-lumbar-spine-degenerative-classification/nyctr7h1/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/swinv2_tiny_unet_scse_20x256x256_mixup/Sagittal_T2-STIR/fold1/rsna-2024-lumbar-spine-degenerative-classification/pen1py6r/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/convnext_tiny_u

  0%|          | 0/397 [00:00<?, ?it/s]

...

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/395 [00:00<?, ?it/s]

loading ckpt:  ../../../logs/stage1/exp017/swinv2_tiny_unet_scse_20x256x256_mixup/Sagittal_T1/fold4/rsna-2024-lumbar-spine-degenerative-classification/577d9cb3/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/convnext_tiny_unet_scse_20x256x256_mixup/Sagittal_T1/fold4/rsna-2024-lumbar-spine-degenerative-classification/fka0hdw2/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/caformer_s18_unet_scse_20x256x256_mixup/Sagittal_T1/fold4/rsna-2024-lumbar-spine-degenerative-classification/wvyhss9l/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/resnetrs50_unet_scse_20x256x256_mixup/Sagittal_T1/fold4/rsna-2024-lumbar-spine-degenerative-classification/9kfskiz4/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/swinv2_tiny_unet_scse_20x256x256_mixup/Sagittal_T2-STIR/fold4/rsna-2024-lumbar-spine-degenerative-classification/3tq800c1/checkpoints/best_loss.ckpt
loading ckpt:  ../../../logs/stage1/exp017/convnext_tiny_u

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/395 [00:00<?, ?it/s]

In [5]:
sagittal_t1_pred_df = pd.DataFrame(sagittal_t1_result, columns=["series_id_x", "level", "x_x_pred", "y_x_pred"])
sagittal_t2_pred_df = pd.DataFrame(sagittal_t2_result, columns=["series_id_y", "level", "x_y_pred", "y_y_pred"])

In [6]:
sagittal_t1_pred_df.loc[:, ["series_id_x", "level", "x_x_pred", "y_x_pred"]].to_csv("Sagittal_T1_pred.csv", index=False)
sagittal_t2_pred_df.loc[:, ["series_id_y", "level", "x_y_pred", "y_y_pred"]].to_csv("Sagittal_T2-STIR_pred.csv", index=False)